# LLM Pretrain or Finetune Model Workflow for Model Registry

## Setup

* Create a python3.8 conda env
`conda create --name {your_preferred_env_name} python=3.8`
* And, then install the latest snowparkML python package(minimum 1.0.12)

In [ ]:
!pip install --force-reinstall --no-deps /home/ubuntu/snowml/bazel-bin/snowflake/ml/snowflake_ml_python-1.0.12-py3-none-any.whl

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2


In [23]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
import pandas as pd
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from IPython.display import JSON

Connection config available at ~/.snowsql/config

In [5]:
session = Session.builder.configs(SnowflakeLoginOptions('connections.demo')).create()

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [ ]:
session.get_current_database(), session.get_current_schema()

In [9]:
REGISTRY_DATABASE_NAME = "HALU_MR"
REGISTRY_SCHEMA_NAME = "PUBLIC"

In [10]:

model_registry.create_model_registry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)
registry = model_registry.ModelRegistry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)

# Registry opertions

## Define the model

### Case1: Local Lora Finetune Weights
Lora finetune weights by huggingface PEFT library is supported.

In [14]:
!ls /home/ubuntu/projects/test_ft_weights

adapter_config.json  adapter_model.bin	halu_peft_ft  training_args.bin


In [36]:
options = llm.LLMOptions(
    token="...",
    max_batch_size=100,
)
model = llm.LLM(
    model_id_or_path="/home/ubuntu/projects/test_ft_weights",
    options=options
)

### Case2: Pretrain models

In [17]:
options = llm.LLMOptions(
    token="...",
    max_batch_size=100,  
)
model = llm.LLM(
    model_id_or_path="meta-llama/Llama-2-7b-chat-hf",
    options=options
)

## Log model

In [37]:
svc_model = registry.log_model(
    model_name='llm_notebook_ft',
    model_version='v1',
    model=model,
    options={"embed_local_ml_library": True},
)

## Deploy

In [45]:
from snowflake.ml.model import deploy_platforms

deployment_options = {
    "compute_pool": 'BUILD_2023_POOL',
    "num_gpus": 1,
    "image_repo": 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo',
    "enable_remote_image_build": True,
    "debug_mode": True,
}
        
deploy_info = svc_model.deploy(
    deployment_name="llm_notebook_ft_1",
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True,
    options=deployment_options
)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:353: UserWarning: Found dependencies specified as pip requirements. This may prevent model deploying to Snowflake Warehouse.
  warnings.warn(


## Prediction

In [20]:
PROMPT_TEMPLATE = """

[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys. Start response by "{".
'location': Location of the caller. Include city only.
'toy_list": List of toy names from the caller.

### Input:
"""

def build_prompt(input):
    return PROMPT_TEMPLATE + input + "\n[/INST]"

In [21]:
df = pd.read_json('/home/ubuntu/projects/v8.jsonl', lines=True)

In [27]:
df.iloc[0].to_dict()

{'language': 'EN',
 'transcript': "caller: Hello!\nfrosty: Well, hello! Who's spreading holiday cheer with me today?\ncaller: I'm Max from Sydney.\nfrosty: Hello, Max! Can you tell me what's on your wish list this holiday?\ncaller: Hmm, I am not sure. I guess I like cars.\nfrosty: We have a fun Bluey car. It's very cool. And also, there's a Teenage Mutant Ninja Turtles pizza delivery van! It's really fun.\ncaller: Oh, the bluey car sounds cool.\nfrosty: Great choice, Max! By the way, how do you plan to celebrate the holiday season with your family?\ncaller: We're going to the beach! It's summer here in Sydney.\nfrosty: Oh, that sounds wonderful, Max. So, we will put the Bluey car on your holiday wish list, okay?\ncaller: Yes, please!\nfrosty: It’s all done. I hope your holiday is filled with joy and fun!",
 'name': 'Max',
 'location': 'Sydney',
 'toy_list': ['Bluey Convertible and Figures']}

In [28]:
dfl = df['transcript'].to_list()[:10]

In [29]:
prompts = [build_prompt(t) for t in dfl]

In [30]:
input_df = pd.DataFrame({'input': prompts})

In [40]:
res = svc_model.predict(
    deployment_name='llm_notebook_ft_1',
    data=input_df
)

In [42]:
pd.set_option('display.max_colwidth', None)

In [43]:
res

,generated_text
0,"{""toy_list"": [""Bluey Convertible and Figures"", ""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""], ""location"": ""Sydney""}"
1,"{""toy_list"": [""Furby interactive plush toy"", ""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""London""}"
2,"{""toy_list"": [""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""], ""location"": ""Auckland""}"
3,"{""toy_list"": [""Transformers Rise of the Beasts Beast-Mode Bumblebee""], ""location"": ""Denver""}"
4,"{""toy_list"": [""Fingerlings"", ""Barbie Dreamhouse 2023""], ""location"": ""Sydney""}"
5,"{""toy_list"": [""Barbie Science Lab Playset"", ""Furby interactive plush toy""], ""location"": ""Houston, Texas""}"
6,"{""toy_list"": [""Star Wars LOLA animatronic droid"", ""Bluey Convertible and Figures""], ""location"": ""Sydney""}"
7,"{""toy_list"": [""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van"", ""Bitzee interactive pet""], ""location"": ""Dublin""}"
8,"{""toy_list"": [""Barbie Science Lab Playset""], ""location"": ""Melbourne, Australia""}"
9,"{""toy_list"": [""Sesame Street Monster Meditation Elmo""], ""location"": ""Toronto""}"


In [44]:
svc_model.delete_deployment(deployment_name='llm_notebook_ft_1')